Analysis for cell type frequency across different OR groups (including code for Fig. 4C)

In [ ]:
# Author: Dalit Engelhardt

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sklearn
import random
import glob
import pyreadr
from operator import itemgetter
import itertools
from itertools import groupby
import seaborn as sns
import matplotlib.ticker as ticker
import sys
import scipy.stats  as stats
pd.set_option('display.max_columns', None)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
import statsmodels.api as sm

In [ ]:
tumor_roi_dat = pd.read_csv('new_annotation_Feb2021_KW.txt', delimiter = "\t")
protein_dat=pd.read_csv('ERCC_IgG_norm.csv')
nuc_count = pd.read_csv('nuc_count.csv')

protein_dat=protein_dat.rename({'Unnamed: 0': 'Protein'}, axis=1) 

all_rois=list(protein_dat.columns)[1:]

tumor_core_roi_dict={}
excluded_rois=[]
for loc in range(len(tumor_roi_dat)):
    cur_row = tumor_roi_dat.iloc[loc]
    roi = cur_row['ROI']
    roi_num=roi.split('_')[1]
    if len(roi_num)==1:
        roi = 'ROI_0'+roi_num
    #print(cur_row)
    if roi in all_rois:
        key = (cur_row['Tumor_ID'],cur_row['Core_ID'])
        if key[0]==21 or key[0]==8:
            excluded_rois.append(roi)
        else:
            if key in tumor_core_roi_dict:
                tumor_core_roi_dict[key].append(roi)
            else:
                tumor_core_roi_dict[key] = [roi]            


tumor_ids=list(tumor_roi_dat['Tumor_ID'])
core_ids=list(tumor_roi_dat['Core_ID'])
fish_cores=list(tumor_roi_dat['FISH_core'])

tumor_core_to_shreya_base={(tumor_ids[i],core_ids[i]):fish_cores[i] for i in range(len(tumor_roi_dat))}
tumor_core_to_shreya={}
for k,v in tumor_core_to_shreya_base.items():
    if k in tumor_core_roi_dict.keys():
        tumor_core_to_shreya[k]=v
all_proteins_cores_shreya=list(tumor_core_to_shreya.values())

tumor_roi_dict={}
for k, rois in tumor_core_roi_dict.items():
    tumor=k[0]
    if tumor in tumor_roi_dict.keys():
        tumor_roi_dict[tumor]=tumor_roi_dict[tumor]+rois
    else:
        tumor_roi_dict[tumor]=rois
            
nuc_count=nuc_count.rename({'Unnamed: 0': 'ROI'}, axis=1) 

tumor_protein_dfs=[]
for tumor,rois in tumor_roi_dict.items():
    cur_df=protein_dat[rois]
    cur_df.index=protein_dat['Protein']
    tot_nuc_count=nuc_count[nuc_count['ROI'].isin(protein_dat[rois])]['nuclei_count'].sum()
    for roi in rois:
        cur_weight=list(nuc_count[nuc_count['ROI']==roi]['nuclei_count'])[0]/tot_nuc_count
        cur_df[roi]=cur_df[roi]*cur_weight 
    cur_df=cur_df.sum(axis=1).to_frame().rename(columns={0:tumor})
    tumor_protein_dfs.append(cur_df)
protein_dat_by_tumor=pd.concat(tumor_protein_dfs,1)

protein_dat_by_tumor=protein_dat_by_tumor.T

In [ ]:
filenames = glob.glob("FISH_data/table*")
dfs_o = {}

for filename in filenames:
    df_o = pd.read_csv(filename)
    shreya_core_num = int(filename.split('_')[2])
    image_num = int(filename.split('_')[3].split('.')[0])
    dfs_o[(shreya_core_num,image_num)]=df_o

    
HI=6
MAX_HI = 50

def geno_hi(geno,df):
    return (df[geno]>=HI)&(df[geno]<=MAX_HI)
def geno_lo(geno,df):
    return (df[geno]>=1)&(df[geno]<HI)

for df_fish in list(dfs_o.values()):
    df_fish['EGFR_amp']=1*(geno_hi('EGFR',df_fish))
    df_fish['PDGFRA_amp']=1*(geno_hi('PDGFRA',df_fish))
    df_fish['CDK4_amp']=1*(geno_hi('CDK4',df_fish))
    
dfs={}    
for key,df in dfs_o.items():

    if key[0] not in tumor_core_to_shreya.values():
        continue

    df['EGFR_CDK4_amp']=1*((df['EGFR']>=HI)&(df['CDK4']>=HI))
    df['EGFR_NOT_CDK4_amp']=1*((df['EGFR']>=HI)&(df['CDK4']<HI))
    df['CDK4_NOT_EGFR_amp']=1*((df['CDK4']>=HI)&(df['EGFR']<HI))    
    df['Non_amp']=1*((df['EGFR']<HI)&(df['CDK4']<HI))
    
    dfs[key]=df[(df['EGFR']<=MAX_HI)&(df['PDGFRA']<=MAX_HI)&(df['CDK4']<=MAX_HI)]


In [ ]:
shreya_to_tumor={v:k[0] for k,v in tumor_core_to_shreya.items()}
core_list=list(set([item[0] for item in dfs.keys()]))
tumor_list = list(set([shreya_to_tumor[core] for core in core_list]))

tumor_image_dict={}
for k in dfs.keys():
    this_tumor=shreya_to_tumor[k[0]]
    if this_tumor in tumor_image_dict.keys():
        tumor_image_dict[this_tumor]=tumor_image_dict[this_tumor]+[k]
    else:
        tumor_image_dict[this_tumor]=[k]
        
dfs_in_tumor={}
for tumor, images in tumor_image_dict.items():
    dfs_in_tumor[tumor]=pd.concat([dfs[image] for image in images])

In [ ]:
#TERT
filenames = glob.glob("hTERTmut_count_results/Log_MAX_TMAtert_TMA_*")
tert_files = {}
xy_files = {}

for filename in filenames:
    file_type = filename.split('TMA')[-1].split('_')[-1].split('.')[0]
    core_image = filename.split('TMA')[-1].split('__')[0].split('_')
    core = int(core_image[1])
    image = int(core_image[2])
    if file_type =='tert':
        tert_files[(core,image)]=filename
    elif file_type == 'xyarea':
        xy_files[(core,image)]=filename
        
dfs_TERT={}
for key in list(tert_files.keys()):
    if key in list(xy_files.keys()):
        
        if key[0] not in tumor_core_to_shreya.values():
            continue
        
        df_tert = pd.read_csv(tert_files[key],header=None)
        df_xy = pd.read_csv(xy_files[key],header=None)

        df_xy=df_xy.set_index(0)
        df_xy=df_xy.rename(columns={1:'X',2:'Y'})
        df_xy = df_xy.drop(3,1)

        df_tert=df_tert.set_index(0)
        df_tert=df_tert.rename(columns={1:'TERT_count'})

        df=pd.concat([df_xy,df_tert],axis=1, sort=False)
        df['TERT_bool']=1*(df['TERT_count']>0)
        if len(df)>10:
            dfs_TERT[key]=df
            
tumor_image_dict_TERT={}
for k in dfs_TERT.keys():
    this_tumor=shreya_to_tumor[k[0]]
    if this_tumor in tumor_image_dict_TERT.keys():
        tumor_image_dict_TERT[this_tumor]=tumor_image_dict_TERT[this_tumor]+[k]
    else:
        tumor_image_dict_TERT[this_tumor]=[k]
        
dfs_in_tumor_TERT={}
for tumor, images in tumor_image_dict_TERT.items():
    dfs_in_tumor_TERT[tumor]=pd.concat([dfs_TERT[image] for image in images])

In [ ]:
tumor_OR_df=pd.read_csv('tumor_OR_df.csv').rename(columns={'Unnamed: 0':'Tumor'})
tumor_OR_df.index=tumor_OR_df['Tumor']
tumor_OR_df=tumor_OR_df.drop(['Tumor'],1)

In [ ]:
dfs_in_tumor.keys()

In [ ]:
len(dfs_in_tumor_TERT.keys())

In [ ]:
set(list(dfs_in_tumor.keys())).difference(set(list(dfs_in_tumor_TERT.keys())))

In [ ]:
EGFR_NOT_CDK4_dict={}
CDK4_NOT_EGFR_dict={}
EGFR_AND_CDK4_dict={}
Non_amp_dict={}
for tumor, this_df in dfs_in_tumor.items():
    EGFR_NOT_CDK4_dict[tumor]=100*len(this_df[this_df['EGFR_NOT_CDK4_amp']==1])/len(this_df)
    CDK4_NOT_EGFR_dict[tumor]=100*len(this_df[this_df['CDK4_NOT_EGFR_amp']==1])/len(this_df)
    EGFR_AND_CDK4_dict[tumor]=100*len(this_df[this_df['EGFR_CDK4_amp']==1])/len(this_df)
    Non_amp_dict[tumor]=100*len(this_df[this_df['Non_amp']==1])/len(this_df)

In [ ]:
TERT_prop_dict={}
for tumor, this_df in dfs_in_tumor_TERT.items():
    TERT_prop_dict[tumor]=100*len(this_df[this_df['TERT_bool']==1])/len(this_df)
TERT_prop_df=pd.DataFrame.from_dict(TERT_prop_dict,orient='index').rename(columns={0:'hTERT'})

In [ ]:
EGFR_NOT_CDK4_df=pd.DataFrame.from_dict(EGFR_NOT_CDK4_dict,orient='index').rename(columns={0:'E'})
CDK4_NOT_EGFR_df=pd.DataFrame.from_dict(CDK4_NOT_EGFR_dict,orient='index').rename(columns={0:'C'})
EGFR_AND_CDK4_df=pd.DataFrame.from_dict(EGFR_AND_CDK4_dict,orient='index').rename(columns={0:'EC'})
Non_amp_df=pd.DataFrame.from_dict(Non_amp_dict,orient='index').rename(columns={0:'NA'})
df_geno_props=pd.concat([EGFR_NOT_CDK4_df,CDK4_NOT_EGFR_df,EGFR_AND_CDK4_df,Non_amp_df],axis=1)

In [ ]:
OR_master=pd.concat([df_geno_props,tumor_OR_df,TERT_prop_df],1)

In [ ]:
df_geno_props.loc[1]

In [ ]:
tumor_OR_df.loc[1]

In [ ]:
OR_dfs=[]
for col in ['E', 'C', 'EC', 'NA','hTERT']:
    this_df=OR_master[['OR_char',col]]
    this_df['Name']=col
    this_df=this_df.rename(columns={col:'Cell frequency (%)'})
    this_df=this_df.sort_values(by='OR_char',ascending=False)
    OR_dfs.append(this_df)
OR_info=pd.concat(OR_dfs)

In [ ]:
OR_info['OR_char'].unique()

In [ ]:
OR_info.to_csv('OR_grouping_and_frequency.csv')

In [ ]:
plt.rcParams["font.family"] = "Arial"
plt.rcParams["font.size"] = "26"
fig = plt.figure(figsize=(12,6),linewidth=10)
ax=sns.boxplot(x="Name", y="Cell frequency (%)",hue="OR_char",palette=["salmon", 'lightgray',"lightsteelblue"], data=OR_info,width=0.7)

plt.xlabel('')

plt.setp(ax.artists, edgecolor = 'black')
plt.setp(ax.lines, color='black')

for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(1.2)
#plt.savefig('prop_distributions_across_OR_groups.pdf')
plt.show()

In [ ]:
plt.rcParams["font.family"] = "Arial"
plt.rcParams["font.size"] = "26"
fig = plt.figure(figsize=(12,6),linewidth=10)
ax=sns.boxplot(x="Name", y="Cell frequency (%)",hue="OR_char",palette=["salmon", 'lightgray',"lightsteelblue"], data=OR_info,width=0.7)

plt.xlabel('')

plt.setp(ax.artists, edgecolor = 'black')
plt.setp(ax.lines, color='black')

ax.legend_.remove()
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(1.2)
#plt.savefig('prop_distributions_across_OR_groups_reordered.pdf')
plt.show()

In [ ]:
def get_MW_stats(dist1,dist2):
    MW_results=stats.mannwhitneyu(dist1, dist2)
    MW_statistic=MW_results.statistic
    MW_pval=MW_results.pvalue
    return [MW_statistic,MW_pval]

In [ ]:
dist1=list(OR_master[OR_master['OR_char']=='OR, bottom 1/3']['E'])
dist2=list(OR_master[OR_master['OR_char']=='OR, top 1/3']['E'])
get_MW_stats(dist1,dist2)

In [ ]:
dist1=list(OR_master[OR_master['OR_char']=='OR, bottom 1/3']['C'])
dist2=list(OR_master[OR_master['OR_char']=='OR, top 1/3']['C'])
get_MW_stats(dist1,dist2)

In [ ]:
dist1=list(OR_master[OR_master['OR_char']=='OR, bottom 1/3']['EC'])
dist2=list(OR_master[OR_master['OR_char']=='OR, top 1/3']['EC'])
get_MW_stats(dist1,dist2)

In [ ]:
dist1=list(OR_master[OR_master['OR_char']=='OR, bottom 1/3']['NA'])
dist2=list(OR_master[OR_master['OR_char']=='OR, top 1/3']['NA'])
get_MW_stats(dist1,dist2)

In [ ]:
dist1=list(OR_master[OR_master['OR_char']=='OR, bottom 1/3']['hTERT'])
dist2=list(OR_master[OR_master['OR_char']=='OR, top 1/3']['hTERT'])
get_MW_stats(dist1,dist2)